In [2]:
%pip install datasets torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
from typing import List
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = torch.device("mps")
eval_iters = 200

torch.manual_seed(1337)

/Users/mateja/Projects/gpt-sandbox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_data = load_dataset("tiny_shakespeare", split="train")[0]["text"]
val_data = load_dataset("tiny_shakespeare", split="validation")[0]["text"]

vocab = sorted(list(set(train_data + val_data)))
vocab_size = len(vocab)

stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for i, ch in enumerate(vocab)}

In [5]:
def encode(s: str):
    return [stoi[c] for c in s]


def decode(l: List):
    return "".join([itos[i] for i in l])

In [6]:
train_data = torch.tensor(encode(train_data), dtype=torch.long)
val_data = torch.tensor(encode(val_data), dtype=torch.long)

In [7]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size, device=device)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # B T C

        if targets is None:
            loss = None
        else:
            # Reshape logits
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [10]:
model = BigramLanguageModel(vocab_size)
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iters):
    if i % eval_interval == 0:
        losses = estimate_loss()
        print(
            f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # Sample a batch of data
    xb, yb = get_batch("train")

    # Evaluate the losses
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.6642, val loss 4.6523
step 300: train loss 2.8183, val loss 2.8177
step 600: train loss 2.5413, val loss 2.5697
step 900: train loss 2.5003, val loss 2.5058
step 1200: train loss 2.4920, val loss 2.4943
step 1500: train loss 2.4707, val loss 2.4916
step 1800: train loss 2.4567, val loss 2.4863
step 2100: train loss 2.4614, val loss 2.4857
step 2400: train loss 2.4652, val loss 2.4882
step 2700: train loss 2.4586, val loss 2.4841
2.459425210952759


Towhaprse tize herst e.
O d fZEie hy:


Hak, CORineg aggell ther Masearor charnge?
Ten,
re thy, chouspo in mpery way avend oubur'er sinkes bokecard dhiceny

He tw el fe oupise he, lbustselownthous;
I m w
T:
The at;
I m hofaruk mondrn itheland's oe, oghithet f, badogienthofathatey foucouler'dsureisold array n
ICoyockind m wins, in mamybalorenyongmyoorord Vofetthindy hak shil brveseay alsteanerm to, oupomp! wee d pre h, gavitYOrd Thean apsts lathind my d erouerse IOLUEDid nghathicorire.F n IS:
IO


In [11]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [12]:
# Version 1
# We want x[b, t] = mean_{i<=t} x[b, i]
x_bow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        x_prev = x[b, : t + 1]  # (t, C)
        x_bow[b, t] = torch.mean(x_prev, 0)

In [14]:
# Version 2
w = torch.tril(torch.ones(T, T))
w = w / w.sum(1, keepdim=True)
x_bow_2 = w @ x  # (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(x_bow, x_bow_2)

True

In [16]:
# Version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
w = torch.zeros((T, T))
w = w.masked_fill(tril == 0, float("-inf"))
w = F.softmax(w, dim=1)
x_bow_3 = w @ x
torch.allclose(x_bow_2, x_bow_3)

True